# 0 Load packages

In [1]:
import pandas as pd
import numpy as np

# 1 Read DF

## 1.1 Read the given dataset

In [2]:
df = pd.read_csv("interview_signup.csv")

C:\Users\Jakob\AppData\Local\Temp\ipykernel_20580\649760490.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("interview_signup.csv")


In [3]:
len(df)

318345

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318345 entries, 0 to 318344
Data columns (total 5 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   original_product_name  318345 non-null  object 
 1   postcode               318345 non-null  object 
 2   bundesland             288813 non-null  object 
 3   total_bonus            318345 non-null  float64
 4   order_date             318345 non-null  object 
dtypes: float64(1), object(4)
memory usage: 12.1+ MB


In [5]:
df.sample(frac=1).head(10)

,original_product_name,postcode,bundesland,total_bonus,order_date
234697,E.ON STROM 24,27432.0,Niedersachsen,186.0,2018-09-27
67406,E.ON STROM Ã–KO,22457.0,Hamburg,151.0,2018-03-19
124041,E.ON STROM ÖKO,34454.0,Hessen,149.0,2018-05-23
101828,E.ON STROM 24,84095.0,Bayern,161.0,2018-04-28
275172,E.ON STROM ÖKO 24,41068,Nordrhein-Westfalen,264.0,2018-11-12
291485,E.ON STROM,46569,Nordrhein-Westfalen,205.0,2018-12-01
287849,E.ON STROM 24,37412.0,NaN,104.0,2018-11-27
224139,E.ON STROM,64354.0,Hessen,186.0,2018-09-15
23097,E.ON STROM,84160.0,Bayern,115.0,2018-01-27
62704,E.ON STROM ÖKO,52152.0,Nordrhein-Westfalen,156.0,2018-03-13


### Discovered problems

*   Postcode is of datatype objects and has mixed datatypes
*   Bundesland has missing values

## 1.2 Read a PLZ-Bundesland mapping for reference and potential later usage

In [6]:
plz = pd.read_csv("PLZ_Bundesland.csv", usecols = ["PLZ", "Bundesland"])
#plz["PLZ"] = plz["PLZ"].astype("object")

In [7]:
plz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8196 entries, 0 to 8195
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PLZ         8196 non-null   int64 
 1   Bundesland  8196 non-null   object
dtypes: int64(1), object(1)
memory usage: 128.2+ KB


In [8]:
plz.head()

,PLZ,Bundesland
0,1067,Sachsen
1,1069,Sachsen
2,1097,Sachsen
3,1099,Sachsen
4,1108,Sachsen


# 2 Checking each column

## 2.1 Product Name

In [9]:
df["original_product_name"].value_counts(dropna=False)

E.ON STROM                         164473
E.ON STROM 24                       69363
E.ON STROM ÖKO                      55194
E.ON STROM ÖKO 24                   25440
E.ON STROM PUR                       3650
E.ON STROM Ã–KO                       206
E.ON STROM 24 24                       14
E.ON STROM 24 24 24                     2
E.ON STROM 24 24 24 24 24 24 24         2
E.ON STROM ÖO                           1
Name: original_product_name, dtype: int64

### Discovered problems

*   There seem to be inconsistencies in the naming of the product
*   It seems as if there are 5 valid products:

    * E.ON STROM
    * E.ON STROM 24
    * E.ON STROM ÖKO
    * E.ON STROM ÖKO 24
    * E.ON STROM PUR


### Solution for original_product_name

In [10]:
#Create new column
df["original_product_name_cleaned"] = df["original_product_name"]

#Replace everything containing (KO and 24) OR (Ö and 24) with E.ON STROM ÖKO 24
df.loc[df["original_product_name"].str.contains('Ö*24|KO*24', regex=True) ,"original_product_name_cleaned"] = "E.ON STROM ÖKO 24"

#Replace everything containing (KO or Ö) BUT NOT 24 with E.ON STROM ÖKO
df.loc[(df["original_product_name"].str.contains('Ö|KO', regex=True)) & ~(df["original_product_name"].str.contains('24', regex=True)) ,"original_product_name_cleaned"] = "E.ON STROM ÖKO"

#Replace everything containing 24 BUT NOT (Ö or KO) with "E.ON STROM 24"
df.loc[(df["original_product_name"].str.contains('24', regex=True)) & ~(df["original_product_name"].str.contains('Ö|KO', regex=True)) ,"original_product_name_cleaned"] = "E.ON STROM 24"


In [11]:
df["original_product_name_cleaned"].value_counts(dropna=False)

E.ON STROM           164473
E.ON STROM 24         69381
E.ON STROM ÖKO        55401
E.ON STROM ÖKO 24     25440
E.ON STROM PUR         3650
Name: original_product_name_cleaned, dtype: int64

## 2.2 Post Code

In [12]:
df["postcode"].value_counts(dropna=False, normalize=True)

31515.0    0.001043
28844.0    0.000964
40764.0    0.000876
28816.0    0.000870
30880.0    0.000854
             ...   
91583      0.000003
67814      0.000003
68199.0    0.000003
24226.0    0.000003
21444      0.000003
Name: postcode, Length: 20525, dtype: float64

In [13]:
#Are special characters are used in postcode?
df.loc[df["postcode"].astype("str").str.contains("[^A-Za-z0-9]", regex=True) , "postcode"].value_counts(dropna=False, normalize=True)

31515.0    0.001203
28844.0    0.001112
40764.0    0.001011
28816.0    0.001004
30880.0    0.000985
             ...   
29581.0    0.000004
46487.0    0.000004
93471.0    0.000004
76889.0    0.000004
77799.0    0.000004
Name: postcode, Length: 13402, dtype: float64

In [14]:
#Are other letters used in postcode?
df.loc[df["postcode"].astype("str").str.contains("[^.0-9]", regex=True) , "postcode"].value_counts(dropna=False, normalize=True)

92696JAVAS    1.0
Name: postcode, dtype: float64

### Discovered problems: 

* mixed data format for postcode with floats and other characters

### Solution for postcode

In [15]:
#Convert postcode to Integer
df["postcode_int"] = df["postcode"].astype("str").str.replace("[a-zA-Z]",'', regex=True).replace("[^.0-9]",'', regex=True).astype("float").astype("int")

In [16]:
print(df["postcode_int"].describe())

count    318345.000000
mean      52096.039344
std       26828.201838
min        1067.000000
25%       30966.000000
50%       50374.000000
75%       74366.000000
max       99998.000000
Name: postcode_int, dtype: float64


### Postcodes now look plausible but for safety let's also check if they are all valid:

In [17]:
df.merge(plz, how="left", left_on="postcode_int", right_on="PLZ").query("PLZ.isnull()")

,original_product_name,postcode,bundesland,total_bonus,order_date,original_product_name_cleaned,postcode_int,PLZ,Bundesland
7831,E.ON STROM ÖKO,98694.0,Thüringen,130.0,2018-01-09,E.ON STROM ÖKO,98694,NaN,NaN
25220,E.ON STROM ÖKO,98694.0,Thüringen,189.0,2018-01-29,E.ON STROM ÖKO,98694,NaN,NaN
29938,E.ON STROM,98694.0,Thüringen,129.0,2018-02-04,E.ON STROM,98694,NaN,NaN
46823,E.ON STROM ÖKO,64760.0,Hessen,108.0,2018-02-23,E.ON STROM ÖKO,64760,NaN,NaN
52774,E.ON STROM ÖKO,98694.0,Thüringen,147.0,2018-03-02,E.ON STROM ÖKO,98694,NaN,NaN
52969,E.ON STROM ÖKO,39628.0,NaN,162.0,2018-03-02,E.ON STROM ÖKO,39628,NaN,NaN
73615,E.ON STROM ÖKO 24,60312.0,NaN,84.0,2018-03-26,E.ON STROM ÖKO 24,60312,NaN,NaN
84393,E.ON STROM 24,64760.0,Hessen,171.0,2018-04-08,E.ON STROM 24,64760,NaN,NaN
87035,E.ON STROM ÖKO,39628.0,NaN,108.0,2018-04-11,E.ON STROM ÖKO,39628,NaN,NaN
122183,E.ON STROM,98694.0,Thüringen,118.0,2018-05-21,E.ON STROM,98694,NaN,NaN


--> Seems like only a small fraction of PLZs were not in the database, the large majority of the PLZs seem to be in the database

## 2.3 bundesland

In [18]:
#df.groupby("bundesland", dropna=False).size().sort_values(ascending=False)
df["bundesland"].value_counts(dropna=False, normalize=True).sort_values(ascending=False)

Nordrhein-Westfalen       0.215304
Bayern                    0.159390
Baden-Württemberg         0.111348
Niedersachsen             0.109582
NaN                       0.092767
Hessen                    0.062505
Rheinland-Pfalz           0.051353
Schleswig-Holstein        0.040714
Brandenburg               0.027640
Sachsen                   0.027147
Berlin                    0.025953
Thüringen                 0.018760
Mecklenburg-Vorpommern    0.017214
Sachsen-Anhalt            0.013994
Hamburg                   0.012417
Saarland                  0.009389
Bremen                    0.004523
Name: bundesland, dtype: float64

### Discovered problems

*   9.3% of the records have missing data for bundesland

### Solution for bundesland:

infer Bundesland from other postcodes or infer Bundesland from external database

## 2.4 total bonus


In [19]:
df["total_bonus"].describe()

count    318345.000000
mean        150.590275
std          53.692176
min           0.000000
25%         112.000000
50%         149.000000
75%         182.000000
max         570.000000
Name: total_bonus, dtype: float64

### There seem to be no problems with the total_bonus field

## 2.4 order_date

In [20]:
df["order_date"].value_counts(dropna=False, normalize=True).sort_values(ascending=False)

2018-09-05    0.003016
2018-10-13    0.003006
2018-10-21    0.003006
2018-08-26    0.002962
2018-12-19    0.002953
                ...   
2018-08-28    0.002544
2018-03-18    0.002544
2018-04-25    0.002519
2018-03-10    0.002519
2018-05-23    0.002504
Name: order_date, Length: 365, dtype: float64

### There seem to be no problems with the contents of the date field, but it could be parsed to a pandas datetime format

In [21]:
df["order_date"] = pd.to_datetime(df["order_date"], errors="raise")

# 3 Summary

### The following problems have been discovered

* **original product name**
    * Wrong product names --> Solution: use regex to replace wrong product names

* **postcode**
    * Mixed data format and additional strings found in PLZ --> Solution: remove strings and then convert to Integer format. Then convert to String to avoid numeric interpretation of column

* **bundesland**
    * 9.3% of the records have missing data --> Solution: infer Bundesland from other postcodes or infer Bundesland from external database

* **total_bonus**
    * No problems visible

* **order_date**
    * No problems visible, but it could be parsed to a pandas datetime format

# Go to 02_Using_DataCleaner_Module.ipynb for solution in production